get data from [here](https://www.kaggle.com/competitions/titanic/data)

In [1]:
import csv
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
PASSENGERS_SURVIVED_FILE = "./gender_submission.csv"
TRAIN_FILE = "./train.csv"
TEST_FILE = "./test.csv"

In [4]:
train_labels = []
train_values = []

with open(TRAIN_FILE, "r") as csvfile:
  reader = csv.reader(csvfile)
  next(reader)
  # 0-3 PasengerId, ((Survived)), [Pclass], Name
  # 4-7 [Sex], [Age], [SibSp], [Parch]
  # 8-11 Ticket [Fare] Cabin Embarked

  def present(val):
    return val != '' and val is not None

  def sex(val):
    return 0 if val == 'female' else 1

  def flo(val):
    return float(val)

  def age(val):
    return float(val) if present(val) else 30.3


  for row in reader:
    train_labels.append(int(row[1]))
    train_values.append([
        int(row[2]), # PClass
        sex(row[4]), # Sex
        age(row[5]), # Age
        int(row[6]), # SipSp
        int(row[7]), # Parch
        flo(row[9]), # Fare
    ])
print(np.array(train_values).shape)

(891, 6)


In [7]:
validation_labels = []
validation_values = []
validation_ids = []

with open(TEST_FILE, "r") as csvfile:
  reader = csv.reader(csvfile)
  next(reader)
  # 0-3 Passenger, Pclass, Name, Sex
  # 4-7 Age, Sibsp, Parch, Ticket
  # 8-10 Fare Cabin Embarked

  def present(val):
    return val != '' and val is not None

  def sex(val):
    return 0 if val == 'female' else 1

  def flo(val):
    return float(val) if present(val) else 35.6

  def age(val):
    return float(val) if present(val) else 30.3


  for row in reader:
    validation_ids.append(row[0])
    validation_values.append([
        int(row[1]), # PClass
        sex(row[3]), # Sex
        age(row[4]), # Age
        int(row[5]), # SipSp
        int(row[6]), # Parch
        flo(row[8]), # Fare
    ])

with open(PASSENGERS_SURVIVED_FILE, "r") as csvfile:
  reader = csv.reader(csvfile)
  next(reader)

  for row in reader:
    validation_labels.append(int(row[1]))
print(np.array(validation_values).shape)
print(np.array(validation_ids).shape)
print(np.array(validation_labels).shape)

(418, 6)
[3, 1, 34.5, 0, 0, 7.8292]
(418,)
892
(418,)
0


In [12]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, input_shape=(6,), activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


optimizer = tf.keras.optimizers.Adam()

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy'],
)

history = model.fit(
    x=train_values,
    y=train_labels,
    epochs=35,
    validation_data=(validation_values, validation_labels)
)

Epoch 1/35
28/28 [==============================] - 2s 18ms/step - loss: 1.3613 - accuracy: 0.5690 - val_loss: 1.1046 - val_accuracy: 0.6268
Epoch 2/35
28/28 [==============================] - 0s 10ms/step - loss: 0.6735 - accuracy: 0.6723 - val_loss: 0.6219 - val_accuracy: 0.6675
Epoch 3/35
28/28 [==============================] - 0s 10ms/step - loss: 0.5984 - accuracy: 0.6835 - val_loss: 0.6500 - val_accuracy: 0.6699
Epoch 4/35
28/28 [==============================] - 0s 7ms/step - loss: 0.6261 - accuracy: 0.6655 - val_loss: 0.6841 - val_accuracy: 0.6746
Epoch 5/35
28/28 [==============================] - 0s 7ms/step - loss: 0.6077 - accuracy: 0.6958 - val_loss: 0.7228 - val_accuracy: 0.6746
Epoch 6/35
28/28 [==============================] - 0s 5ms/step - loss: 0.5906 - accuracy: 0.7015 - val_loss: 0.6405 - val_accuracy: 0.6794
Epoch 7/35
28/28 [==============================] - 0s 6ms/step - loss: 0.5762 - accuracy: 0.6992 - val_loss: 0.6301 - val_accuracy: 0.6962
Epoch 8/35
28/28 

In [24]:
# to_predict = np.array(validation_values)
# to_predict = np.expand_dims(to_predict, axis=-1)

with open("./submission.csv", mode="w", newline='') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(["PassengerId", "Survived"])
  predictions = model.predict(validation_values)
  for i in range(len(predictions)):
    prediction = 0 if predictions[i] < 0.5 else 1
    writer.writerow([validation_ids[i], prediction])
  # predictions = list(map(lambda p: 0 if p < 0.5 else 1, predictions))
  # print(predictions[:10])
  # for i in range(len(validation_values)):
  #   prediction = model.predict(to_predict[i])
  #   writer.writerow([validation_ids[i], prediction[0]])

  #   if i == 10:
  #     break


14/14 [==============================] - 0s 2ms/step


In [14]:
print(round(0.2))
print(round(0.6))

0
1
